In [ ]:
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (10, 4)

from env_function import environmental_variables

bbox = (-83.0, 27.3, -82.4, 28.0)  # Tampa Bay
start = "2024-12-01"
end   = "2024-12-31"

env = environmental_variables(bbox, start, end, ["sst", "precip"])

# Immediately load SST and precip once, then reuse these variables
sst = env["sst"].load()
precipipitation = env["precip"].load()

print(sst.sizes, precip.sizes)


In [ ]:
bbox = (-82.80, 27.30, -82.40, 27.70)  # Tampa Bay
start = "2024-12-21"
end = "2024-12-31"

env_data = environmental_variables(bbox, start, end, ["sst", "precip"])
sst = env_data["sst"]
precip = env_data["precip"]

print(sst.time.values[:5], sst.sizes)


In [ ]:
# Plot daily SST and precip side by side for Dec 2024 (fast)

fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=False)

# Left: SST
axes[0].plot(df_dec["time"], df_dec["SST"], color="tab:red", marker="o")
axes[0].set_title("Tampa Bay SST (Dec 2024)")
axes[0].set_xlabel("Time")
axes[0].set_ylabel("SST (°C)")

# Right: Precip
axes[1].plot(df_dec["time"], df_dec["PRECT"], color="tab:blue", marker="s")
axes[1].set_title("Tampa Bay Precip (Dec 2024)")
axes[1].set_xlabel("Time")
axes[1].set_ylabel("Precipitation (mm/day)")

plt.tight_layout()
plt.show()


In [ ]:
# Plot daily SST and precip for Dec 2024 (Tampa Bay)

fig, ax1 = plt.subplots(figsize=(10, 4))

# SST (left y-axis)
sst.plot(ax=ax1, color="tab:red", marker="o", label="SST")
ax1.set_xlabel("Time")
ax1.set_ylabel("SST (°C)", color="tab:red")
ax1.tick_params(axis="y", labelcolor="tab:red")

# Precip (right y-axis)
ax2 = ax1.twinx()
precip.plot(ax=ax2, color="tab:blue", marker="s", label="Precip")
ax2.set_ylabel("Precipitation (mm/day)", color="tab:blue")
ax2.tick_params(axis="y", labelcolor="tab:blue")

plt.title("Tampa Bay SST and Precipitation (Dec 2024)")
fig.tight_layout()
plt.show()


In [ ]:
# cell one 
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from env_function import environmental_variables
import analysis  # your new module

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (10, 4)

bbox = (-83.0, 27.3, -82.4, 28.0)
start = "2019-01-01"
end   = "2024-12-31"

env = environmental_variables(bbox, start, end, ["sst", "precip"])
sst = env["sst"]
precip = env["precip"]

# from Grace's code:
# ndwi_daily, ndci_daily, ndti_daily already computed for same bbox and dates


In [ ]:
# Cell 2: monthly and seasonal means using analysis.py

sst_aggr    = analysis.compute_monthly_seasonal_means(sst)
precip_aggr = analysis.compute_monthly_seasonal_means(precip)
ndwi_aggr   = analysis.compute_monthly_seasonal_means(ndwi_daily)
ndci_aggr   = analysis.compute_monthly_seasonal_means(ndci_daily)
ndti_aggr   = analysis.compute_monthly_seasonal_means(ndti_daily)

sst_monthly    = sst_aggr["monthly"]
precip_monthly = precip_aggr["monthly"]
ndwi_monthly   = ndwi_aggr["monthly"]
ndci_monthly   = ndci_aggr["monthly"]
ndti_monthly   = ndti_aggr["monthly"]

sst_season    = sst_aggr["seasonal"]
precip_season = precip_aggr["seasonal"]
ndwi_season   = ndwi_aggr["seasonal"]
ndci_season   = ndci_aggr["seasonal"]
ndti_season   = ndti_aggr["seasonal"]


In [ ]:
# Cell 3: time-series and seasonal-cycle plots

analysis.plot_time_series(sst_monthly,    "SST (°C)")
analysis.plot_time_series(precip_monthly, "Precipitation (mm/day)")
analysis.plot_time_series(ndci_monthly,   "NDCI")

analysis.plot_seasonal_cycle(ndci_daily,  "NDCI")
analysis.plot_seasonal_cycle(precip,      "Precipitation")


In [ ]:
# Cell 4: build monthly DataFrames and compute correlation matrix

wqi_df = xr.Dataset({
    "NDWI": ndwi_monthly,
    "NDCI": ndci_monthly,
    "NDTI": ndti_monthly,
}).to_dataframe()

env_df = xr.Dataset({
    "SST":   sst_monthly,
    "PRECT": precip_monthly,
}).to_dataframe()

corr_matrix = analysis.compute_correlation_matrix(wqi_df, env_df)
corr_matrix


In [ ]:
# Cell 5: RMSE examples

rmse_ndci_sst   = analysis.compute_rmse(ndci_monthly.values, sst_monthly.values)
rmse_ndci_prect = analysis.compute_rmse(ndci_monthly.values, precip_monthly.values)
rmse_ndti_sst   = analysis.compute_rmse(ndti_monthly.values, sst_monthly.values)
rmse_ndti_prect = analysis.compute_rmse(ndti_monthly.values, precip_monthly.values)

print("RMSE NDCI–SST:   ", rmse_ndci_sst)
print("RMSE NDCI–PRECT: ", rmse_ndci_prect)
print("RMSE NDTI–SST:   ", rmse_ndti_sst)
print("RMSE NDTI–PRECT: ", rmse_ndti_prect)


In [ ]:
# Cell 6: PCA on combined monthly indices + environmental variables

pca_results = analysis.compute_pca(wqi_df, env_df, n_components=2)
loadings = pca_results["loadings"]
loadings


In [ ]:
# Cell 6b: visualize PCA loadings

analysis.plot_pca_results(loadings)
